**Predicting Water Quality is my first notebook on Kaggle. kindly give any suggestions that make my performance improve**
1. Data Describe
2. EDA
3. Data Cleaning
4. feature engineering
5. model building
6. model evaluation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# reading data using pandas 
data = pd.read_csv("../input/water-potability/water_potability.csv")

# Data Describe

In [ ]:
data.head()

the above data contains null values

In [ ]:
# by using this method we know about dtypes and null values
data.info()

In [ ]:
data.isna().sum()

In [ ]:
#tabular format of statistics of water quality dataset
data.describe()

In [ ]:
data.shape

# EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from scipy import stats
print("=======skewness=========")
for i in data.columns:
    print(str(stats.skew(data[i]))+" : "+i)
print("=======kurtosis=========")
for i in data.columns:
    print(str(stats.kurtosis(data[i]))+" : "+i)

In [ ]:
sns.pairplot(data, hue="Potability", palette="husl");

In [ ]:
data['Potability'].value_counts(normalize=True)

# Data Cleaning

In [ ]:
data.ph = data.ph.fillna(np.mean(data.ph))
data.Sulfate = data.Sulfate.fillna(np.mean(data.ph))
data.Trihalomethanes = data.Trihalomethanes.fillna(np.mean(data.ph))

# Feature Engineering

In [ ]:
features = data.columns[:-1]

In [ ]:
for i in features:
    data[i][data[i]<data[i].quantile(0.25)] = np.mean(data[i])
    data[i][data[i]>data[i].quantile(0.75)] = np.mean(data[i])

In [ ]:
def box_plot(data):
    p=1
    fig = plt.figure(figsize=(14,10))
    for i in data.columns:
        plt.subplot(4,4,p)
        sns.boxplot(x=i,data=data)
        p+=1
    plt.show()

In [ ]:
box_plot(data)

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(data.corr(),annot=True)

# Model Building

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=1/3,random_state=17,stratify=y)

In [ ]:
model = ['logistic regression','knn','decision_tree','random_forest','xgb','svm']
cls=[LogisticRegression(),KNeighborsClassifier(n_neighbors=5),DecisionTreeClassifier(),RandomForestClassifier(),XGBClassifier(),SVC()]
models = pd.DataFrame({'model':model,"classifiers":cls})

In [ ]:
models

In [ ]:
cv_score=[]
for x in cls:
    cv_score.append(cross_val_score(x,X_train,y_train,cv=5).mean())

In [ ]:
print("Model performance/accuracy:")
print("===========================")
for i in range(len(cv_score)):
    print(""+model[i]+" : "+str(cv_score[i]))
    print()

In [ ]:
sns.color_palette("pastel")
sns.barplot(x=model,y=cv_score)
plt.xticks(rotation=60)
plt.show()

In [ ]:
data.describe()

# Model Evaluation